# Tutorial 5 -- Human Detection 
In this tutorial, we will use multithreading to capture both color and depth images. We will then employ the YOLOv11 neural network architecture for human detection. To ensure real-time performance, we will utilize the TensorRT deep learning framework. For more details, visit the following website: https://docs.ultralytics.com/guides/nvidia-jetson/#convert-model-to-tensorrt-and-run-inference

In [1]:
# I have uploaded two YOLO TensorRT models to the LEARN page: the YOLO11L FP16 version and the YOLO11N FP32 version. 
# The first model, 'YOLO11n.engine', runs at a faster speed but has limited detection accuracy. 
# The second model, 'yolo11l_half.engine', is the FP16 version, as indicated by 'half' in its name.
# If you need other versions, please refer to the following link:
# https://docs.ultralytics.com/modes/export/#arguments

# Below is the code I used to convert the YOLO11L FP16 model
# from ultralytics import YOLO
# model = YOLO("yolo11l.pt")  
# model.export(format="engine",half=True)  # FP16
# Note: Generating the 'yolo11l.engine' file may take a long time.

In [6]:
camera.stop()
robot.stop()

### Start the ZED2i Camera system

In [1]:
# You will need to load the YOLO model if you skip the first code block.
from ultralytics import YOLO
model = YOLO("yolo11l_half.engine")

#Start the camera system
import traitlets
import cv2
import numpy as np
import pyzed.sl as sl
import math
import numpy as np
import sys
import math
import threading
from traitlets.config.configurable import SingletonConfigurable

# Define a Camera class that inherits from SingletonConfigurable
class Camera(SingletonConfigurable):
    color_value = traitlets.Any() # monitor the color_value variable
    def __init__(self):
        super(Camera, self).__init__()

        self.zed = sl.Camera()
        # Create a InitParameters object and set configuration parameters
        init_params = sl.InitParameters()
        init_params.camera_resolution = sl.RESOLUTION.VGA #VGA(672*376), HD720(1280*720), HD1080 (1920*1080) or ...
        init_params.depth_mode = sl.DEPTH_MODE.ULTRA  # Use ULTRA depth mode
        init_params.coordinate_units = sl.UNIT.MILLIMETER  # Use meter units (for depth measurements)

        # Open the camera
        status = self.zed.open(init_params)
        if status != sl.ERROR_CODE.SUCCESS: #Ensure the camera has opened succesfully
            print("Camera Open : "+repr(status)+". Exit program.")
            self.zed.close()
            exit(1)

         # Create and set RuntimeParameters after opening the camera
        self.runtime = sl.RuntimeParameters()

        #flag to control the thread
        self.thread_runnning_flag = False

        # Get the height and width
        camera_info = self.zed.get_camera_information()
        self.width = camera_info.camera_configuration.resolution.width
        self.height = camera_info.camera_configuration.resolution.height
        self.image = sl.Mat(self.width,self.height,sl.MAT_TYPE.U8_C4, sl.MEM.CPU)
        self.depth = sl.Mat(self.width,self.height,sl.MAT_TYPE.F32_C1, sl.MEM.CPU)
        self.point_cloud = sl.Mat(self.width,self.height,sl.MAT_TYPE.F32_C4, sl.MEM.CPU) 

    def _capture_frames(self): #For data capturing only

        while(self.thread_runnning_flag==True): #continue until the thread_runnning_flag is set to be False
            if self.zed.grab(self.runtime) == sl.ERROR_CODE.SUCCESS:
                
                # Retrieve Left image
                self.zed.retrieve_image(self.image, sl.VIEW.LEFT)
                # Retrieve depth map. Depth is aligned on the left image
                self.zed.retrieve_measure(self.depth, sl.MEASURE.DEPTH)
    
                self.color_value_BGRA = self.image.get_data()
                self.color_value= cv2.cvtColor(self.color_value_BGRA, cv2.COLOR_BGRA2BGR)
                self.depth_image = np.asanyarray(self.depth.get_data())   
                
    def start(self): #start the data capture thread
        if self.thread_runnning_flag == False: #only process if no thread is running yet
            self.thread_runnning_flag=True #flag to control the operation of the _capture_frames function
            self.thread = threading.Thread(target=self._capture_frames) #link thread with the function
            self.thread.start() #start the thread

    def stop(self): #stop the data capture thread
        if self.thread_runnning_flag == True:
            self.thread_runnning_flag = False #exit the while loop in the _capture_frames
            self.thread.join() #wait the exiting of the thread  
            self.zed.close()

def bgr8_to_jpeg(value):#convert numpy array to jpeg coded data for displaying 
    return bytes(cv2.imencode('.jpg',value)[1])

#create a camera object
camera = Camera()
camera.start() # start capturing the data

WARNING ⚠️ Unable to automatically guess model task, assuming 'task=detect'. Explicitly define task for your model, i.e. 'task=detect', 'segment', 'classify','pose' or 'obb'.
[2025-03-12 12:53:28 UTC][ZED][INFO] Logging level INFO
[2025-03-12 12:53:28 UTC][ZED][INFO] Logging level INFO
[2025-03-12 12:53:28 UTC][ZED][INFO] Logging level INFO
[2025-03-12 12:53:29 UTC][ZED][INFO] [Init]  Depth mode: ULTRA
[2025-03-12 12:53:30 UTC][ZED][INFO] [Init]  Camera successfully opened.
[2025-03-12 12:53:30 UTC][ZED][INFO] [Init]  Camera FW version: 1523
[2025-03-12 12:53:30 UTC][ZED][INFO] [Init]  Video mode: VGA@100
[2025-03-12 12:53:30 UTC][ZED][INFO] [Init]  Serial Number: S/N 35566780
Loading yolo11l_half.engine for TensorRT inference...
Average Depth: 492.2709655761719 mmm

/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:86: RuntimeWarning: overflow encountered in reduce
  return ufunc.reduce(obj, axis, dtype, out, **passkwargs)


Average Depth: nan mm07678222656 mm

/tmp/ipykernel_4135/1103006904.py:119: RuntimeWarning: Mean of empty slice
  average_depth = np.nanmean(trimmed_depth_image_cleaned)


Average Depth: 1257.39453125 mm5 mmm

...

Average Depth: nan mmm271484375 mmmm

/tmp/ipykernel_4135/1103006904.py:119: RuntimeWarning: Mean of empty slice
  average_depth = np.nanmean(trimmed_depth_image_cleaned)


Average Depth: nan mm919921875 mmmmm

/tmp/ipykernel_4135/3393630605.py:119: RuntimeWarning: Mean of empty slice
  average_depth = np.nanmean(trimmed_depth_image_cleaned)


Average Depth: nan mm

### Perform object detection on live video data

In [5]:
import cv2
import ipywidgets.widgets as widgets
from IPython.display import display
import collections
import motors
robot = motors.MotorsYukon(mecanum=False)

#create widgets for the displaying of the image
display_color = widgets.Image(format='jpeg', width='45%') 
display_depth = widgets.Image(format='jpeg', width='45%')  
layout=widgets.Layout(width='100%')

sidebyside = widgets.HBox([display_color, display_depth],layout=layout) #horizontal 
display(sidebyside) #display the widget

tracked_bbox = None
x_center_t = None
y_center_t = None

def moving_average(bbox_history):
    # Compute the average of all the bounding box coordinates
    avg_bbox = np.mean(bbox_history, axis=0)
    return avg_bbox

#callback function, invoked when traitlets detects the changing of the color image
def func(change):
    global tracked_bbox
    global x_center_t
    global y_center_t

    frame = change['new']
    results = model(frame,verbose=False)

    trim=[0,0,0,0]

    conf_threshold = 0.6
    
    for result in results: #the input image is one so only one result avaialbe

        chosen_box = None
        
        for i in range (len(result.boxes.cls)):
            if(result.boxes.cls[i] == 0):  #human subject
                if (result.boxes.conf[i] > conf_threshold): #confident its a human
                    bbox = result.boxes.xyxy[i]
                    cv2.rectangle(frame, (int(bbox[0]), int(bbox[1])), (int(bbox[2]), int(bbox[3])), (255, 0, 0), 2)   

                    if tracked_bbox is None:
                        chosen_box = bbox
                        break;
                    
                    if chosen_box is None:
                        chosen_box = bbox
                    else:
                        x_center_c = (chosen_box[0] + chosen_box[2]) / 2
                        y_center_c = (chosen_box[1] + chosen_box[3]) / 2
                        x_center = (bbox[0] + bbox[2]) / 2
                        y_center = (bbox[1] + bbox[3]) / 2

                        # difference
                        distance_new = math.sqrt((x_center - x_center_t)**2 + (y_center - y_center_t)**2)
                        distance_chosen = math.sqrt((x_center_c - x_center_t)**2 + (y_center_c - y_center_t)**2)

                        if distance_new < distance_chosen:
                            chosen_box = bbox
                        

        if chosen_box is None:
            break;
        tracked_bbox = chosen_box
        trim = [int(tracked_bbox[1]), int(tracked_bbox[3]), int(tracked_bbox[0]), int(tracked_bbox[2])]
        trim[0] = int(tracked_bbox[1])
        trim[1] = int(tracked_bbox[3])
        trim[2] = int(tracked_bbox[0])
        trim[3] = int(tracked_bbox[2])

        x_center_t = (tracked_bbox[0] + tracked_bbox[2]) / 2
        y_center_t = (tracked_bbox[1] + tracked_bbox[3]) / 2
        
        # Draw a circle at the center of the bounding box
        radius = 10  # You can adjust the radius as needed
        color = (0, 255, 0)  # Circle color (Green in BGR format)
        thickness = 2  # Circle border thickness
        cv2.circle(frame, (int(x_center_t), int(y_center_t)), radius, color, thickness)
        
    #Scaling is necessary for real-time data display.
    scale = 0.1 

    # Resize the frame for display
    resized_image = cv2.resize(frame, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)
    display_color.value = bgr8_to_jpeg(resized_image)
    
    # Process depth image (apply colormap)
    depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(camera.depth_image , alpha=0.03), cv2.COLORMAP_JET)
    
    # Trim the depth image based on tracked bounding box coordinates
    depth_colormap[:trim[0], :] = 0
    depth_colormap[trim[1]:, :] = 0
    depth_colormap[:, :trim[2]] = 0
    depth_colormap[:, trim[3]:] = 0


    # Resize the depth colormap for display
    resized_depth_colormap = cv2.resize(depth_colormap, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)
    display_depth.value = bgr8_to_jpeg(resized_depth_colormap)

    if x_center_t is not None and y_center_t is not None:
        # Now calculate the average depth of the entire depth image (before trimming and resizing)
        # Trim depth image to the region of interest
        trimmed_depth_image = camera.depth_image[trim[0]:trim[1], trim[2]:trim[3]]
        
        # Remove NaN and invalid depth values (if necessary)
        trimmed_depth_image_cleaned = np.nan_to_num(trimmed_depth_image, nan=0.0).astype(np.float32)
        
        # Optionally, replace zero values with NaN (if they represent no depth or invalid data)
        trimmed_depth_image_cleaned[trimmed_depth_image_cleaned == 0] = np.nan
        
        # Calculate the average depth from the trimmed depth image
        average_depth = np.nanmean(trimmed_depth_image_cleaned)
        
        # Print the average depth
        print(f"Average Depth: {average_depth} mm", end='\r')
    
        if x_center_t > 362:
            robot.right(0.2)
        elif x_center_t < 310:
            robot.left(0.2)
        elif average_depth > 1800: # TODO cut off the closest and highest values from the image first e.g. >2000 and <200
            robot.forward(0.5)
        else:
            robot.stop()
    
    

    # scale = 0.1
    # resized_image = cv2.resize(camera.color_value, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)
    # display_color.value = bgr8_to_jpeg(resized_image)
    
    # camera.depth_image = np.asanyarray(camera.depth.get_data())

    # depth_image_test = camera.depth_image.copy()  #copy the depth image
    # depth_image_test = np.nan_to_num(depth_image_test, nan=0.0).astype(np.float32)  # Change NaN value to 0
    # depth_image_test[:trim[0],:] =0
    # depth_image_test[trim[1]:,:]=0
    # depth_image_test[:,:trim[2]]=0
    # depth_image_test[:,trim[3]:]=0

    # depth_image_test[depth_image_test<200]=0
    # depth_image_test[depth_image_test>1000]=1000
    # depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image_test, alpha=0.03), cv2.COLORMAP_JET)
    # # if(depth_image_test.max() == 0):
    # #     robot.backward(1) # Turn back
    # #     cv2.putText(depth_colormap, 'warning!!!', (336,188), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
    # # else:
    # #     left_side=depth_image_test[:, :336]
    # #     left_side_distance = left_side[left_side!=0].min()
    # #     right_side=depth_image_test[:, 337:]
    # #     right_side_distance = right_side[right_side!=0].min()

    # #     if (left_side_distance < right_side_distance and left_side_distance < 400):
    # #         robot.right(0.7)
    # #         cv2.putText(depth_colormap, 'warning!!!', (336,188), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
    # #     elif(right_side_distance < left_side_distance and right_side_distance < 400):
    # #         robot.left(0.7) #turn left at full speed
    # #         cv2.putText(depth_colormap, 'warning!!!', (336,188), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2, cv2.LINE_AA)
    # #     else:
    # #         robot.forward(0.4) #move forward at half speed
        
    # resized_depth_colormap=cv2.resize(depth_colormap, None, fx=scale, fy=scale, interpolation=cv2.INTER_AREA)
    # display_depth.value = bgr8_to_jpeg(resized_depth_colormap)
    
camera.observe(func, names=['color_value'])
# camera.stop()

## Tasks

1. Please try to calculate the distance between the detected human and the robot using the depth image. (Note: You can refer to Tutorial 2 to obtain the depth information for a specific point.) 

2. Please try to add a collision avoidance function to this program to protect the robot.  

3. Think about how to control the robot so that it moves towards the detected human. 